In [29]:
import sys
sys.path.append('../../..')
from Library.fourierABS import AnalyticalLossFunctionAbs
from scipy.optimize import minimize
from scipy.integrate import quad, dblquad, IntegrationWarning
import warnings
import timeit
import numpy as np

import scipy
import scipy.stats


warnings.filterwarnings('error')

Here, we use Fourier Transform to approximate different expectations in the constraints. More generally, if a function $f$ verifies certain conditions (see Analysis of Fourier transform valuation formulas and applications), we have the following:
$$ \mathbb{E}\left[f(X - m)\right] = \frac{1}{2\pi} \int_{-\infty}^{\infty} \exp((iu - R)m) \mathbf{M}_X(R-iu)\hat{f}(u+iR)$$
where $\mathbf{M}_X$ is the generating function of $X$. 

For $f(x) = (x^+)^2$, $\hat{f}(u + iR) = \frac{2}{i(u + iR)^3}$. For the cross terms, namely, $f(x,y) = x^+ y^+$, 
$\hat{f}(u + iR) = \frac{1}{u_1 + i R_1}\frac{1}{u_2 + i R_2}$

In [30]:

def fourier_integral1d(cplx_integrand, j, x, eta):
    def real_integrand(u):
        return np.real(cplx_integrand(u, j, x, eta))

    real_quad = quad(real_integrand, -np.inf, np.inf, epsrel=1e-4)[0]
    return real_quad

def fourier_integral2d(cplx_integrand, j, k, x, y, eta):
    def real_integrand(u, v):
        return np.real(cplx_integrand(u, v, j, k, x, y, eta))

    real_quad = dblquad(real_integrand, 
                        -np.inf, np.inf,
                        lambda x: -np.inf, lambda x: np.inf, 
                        epsrel=1e-4)[0]
    return real_quad

In [31]:
class FourierGaussLossFunction(AnalyticalLossFunctionAbs):
    def __init__(self, mu, sigma, alpha, c=None):
        self.__mu = np.array(mu)
        self.__sigma = np.array(sigma).reshape((len(mu), len(mu)))
        self.__alpha = alpha
        super(FourierGaussLossFunction, self).__init__(len(mu), c)

    def moment_generating(self, t, i):
        mu = self.__mu[i]
        sigma2 = self.__sigma[i, i]
        log_part = mu * t + 0.5 * sigma2 * t**2
        return np.exp(log_part)

    
    def moment_generating2D(self, vec_t, i, j):
        mu = self.__mu[[i, j]]
        sigma2 = [[self.__sigma[i, i], self.__sigma[i, j]],
                  [self.__sigma[j, i], self.__sigma[j, j]]]
        log_part = np.dot(mu, vec_t)
        log_part += 0.5 * np.dot(vec_t, np.dot(sigma2, vec_t))
        return np.exp(log_part)
    
    def e(self, m):
        return (self.__mu - m).sum()
    
    #this corresponds to the integrand when transforming the expectation corresponding to (x^+)**2
    def g_fourier_integrand(self, u, j, m_j, eta):
        i = 1j
        eta_m_iu = eta - i * u
        res = np.exp(-eta_m_iu * m_j)
        res *= self.moment_generating(eta_m_iu, j)
        res /= i * (u + i * eta) ** 3
        return res
 


    def g(self, i, m_i):
        continue_bool = True
        eta = 1.5 * np.random.rand()
        while continue_bool:
            try:
                integral = fourier_integral1d(self.g_fourier_integrand, i, m_i, eta)
                continue_bool = False
                return 1. / np.pi * integral
            except scipy.integrate.IntegrationWarning:
                #print "g not converging for x = %s, eta = %s" % (m_i, eta)
                eta = 1.5 * np.random.rand()

    
    #This corresponds to the integrand of the cross terms
    def h_fourier_integrand(self, u, v, j, k, x, y, eta):
        i = 1j
        eta_m_iu = eta - i * np.array([u, v])
        res = np.exp(np.dot(-eta_m_iu, [x, y]))
        res *= self.moment_generating2D(eta_m_iu, j, k)
        res /= (u + i*eta[0])**2 * (v + i*eta[1])**2
        return res
    

    
    def h(self, i, j, m_i, m_j):
        continue_bool = True
        eta = 1.5 * np.random.rand(2)
        while continue_bool:
            try:            
                integral = fourier_integral2d(self.h_fourier_integrand, i, j, m_i, m_j, eta)
                continue_bool = False
                return (1 / (2 * np.pi)**2) * integral
            except scipy.integrate.IntegrationWarning:
                eta = 1.5 * np.random.rand(2)
                

    def jac_g_fourier_integrand(self, u, j, m_j, eta):
        i = 1j
        eta_m_iu = eta - i * u
        res = np.exp(-eta_m_iu * m_j)
        res *= self.moment_generating(eta_m_iu, j)
        res /= (u + i * eta)**2
        return res
    
    

    def jac_g(self, i, m_i):
        continue_bool = True
        eta = 1.5 * np.random.rand()
        while continue_bool:
            try:
                integral = fourier_integral1d(self.jac_g_fourier_integrand, i, m_i, eta)
                continue_bool = False
                return (-0.5 / np.pi) * integral
            except scipy.integrate.IntegrationWarning:
                #print "f not converging for x = %s, alpha = %s" % (m_i, eta)
                eta = 1.5 * np.random.rand()
                

    def jac_h_fourier_integrand(self, u, v, j, k, x, y, eta):
        i = 1j
        eta_m_iu = eta - i * np.array([u, v])
        res = np.exp(np.dot(-eta_m_iu, [x, y]))
        res *= self.moment_generating2D(eta_m_iu, j, k)
        res /= (u + i * eta[0]) ** 2 * (-eta_m_iu[1])
        return res

    def jac_h(self, i, j, m_i, m_j):
        continue_bool = True
        eta = 1.5 * np.random.rand(2)
        while continue_bool:
            try:            
                integral = fourier_integral2d(self.jac_h_fourier_integrand, i, j, m_i, m_j, eta)
                continue_bool = False
                return (1 / (2 * np.pi)**2) * integral
            except IntegrationWarning:
                eta = 1.5 * np.random.rand(2)
                
                
    def shortfall_risk(self, m=None):
        m = self._check_argument(m)
        sum_e = self.e(m)
        sum_g, sum_h = 0., 0.
        for i, m_i in enumerate(m):
            sum_g += self.g(i, m_i)
            if self.__alpha != 0:
                for j, m_j in enumerate(m):
                    if j > i:
                        sum_h += self.h(i, j, m_i, m_j)  
        return sum_e + 0.5 * sum_g + self.__alpha * sum_h
    

    
    def shortfall_risk_jac(self, m):
        m = self._check_argument(m)
        res = []        
        for i, m_i in enumerate(m):
            partial_der = 1 + self.jac_g(i, m_i)
            if self.__alpha != 0:
                for j, m_j in enumerate(m):
                    if i != j:
                        partial_der += self.__alpha * self.jac_h(j, i, m_j, m_i)
            res.append(partial_der)
        return np.array(res)
    

In [39]:
#Case: rho = -0.9, alpha = 1

rho = -0.9

mu = [0., 0.]
sigma = [[1., rho], [rho, 1.]]

c = 1.
alpha  = 1.
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})

%timeit res = minimize(loss.objective, init, jac=loss.objective_jac, constraints=cons, method='SLSQP',options={'maxiter': maxiter})
print(res)

23.1 s ± 2.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
     fun: -0.2855887897197693
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([-0.1427944 , -0.14279439])


In [40]:
#Case: rho = -0.5, alpha = 1

rho = -0.5

mu = [0., 0.]
sigma = [[1., rho], [rho, 1.]]

c = 1.
alpha  = 1.
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})



#r to specify how many times to repeat and n number of times the function is called
%timeit res = minimize(loss.objective, init, jac=loss.objective_jac, constraints=cons, method='SLSQP',options={'maxiter': maxiter})
print(res)


11.3 s ± 2.33 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
     fun: -0.2855887897197693
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([-0.1427944 , -0.14279439])


In [41]:
#Case: rho = -0.2, alpha = 1

rho = -0.2

mu = [0., 0.]
sigma = [[1., rho], [rho, 1.]]

c = 1.
alpha  = 1.
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})



#r to specify how many times to repeat and n number of times the function is called
%timeit res = minimize(loss.objective, init, jac=loss.objective_jac, constraints=cons, method='SLSQP',options={'maxiter': maxiter})
print(res)

9.59 s ± 881 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
     fun: -0.2855887897197693
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([-0.1427944 , -0.14279439])


In [42]:
#Case: rho = 0, alpha = 1

rho = 0

mu = [0., 0.]
sigma = [[1., rho], [rho, 1.]]

c = 1.
alpha  = 1.
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})



#r to specify how many times to repeat and n number of times the function is called
%timeit res = minimize(loss.objective, init, jac=loss.objective_jac, constraints=cons, method='SLSQP',options={'maxiter': maxiter})
print(res)

8.79 s ± 1.65 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
     fun: -0.2855887897197693
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([-0.1427944 , -0.14279439])


In [43]:
#Case: rho = 0.2, alpha = 1

rho = 0.2

mu = [0., 0.]
sigma = [[1., rho], [rho, 1.]]

c = 1.
alpha  = 1.
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})



#r to specify how many times to repeat and n number of times the function is called
%timeit res = minimize(loss.objective, init, jac=loss.objective_jac, constraints=cons, method='SLSQP',options={'maxiter': maxiter})
print(res)

9.74 s ± 1.05 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
     fun: -0.2855887897197693
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([-0.1427944 , -0.14279439])


In [44]:
#Case: rho = 0.5, alpha = 1

rho = 0.5

mu = [0., 0.]
sigma = [[1., rho], [rho, 1.]]

c = 1.
alpha  = 1.
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})



#r to specify how many times to repeat and n number of times the function is called
%timeit res = minimize(loss.objective, init, jac=loss.objective_jac, constraints=cons, method='SLSQP',options={'maxiter': maxiter})
print(res)

10.2 s ± 1.48 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
     fun: -0.2855887897197693
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([-0.1427944 , -0.14279439])


In [45]:
#Case: rho = 0.9, alpha = 1

rho = 0.9

mu = [0., 0.]
sigma = [[1., rho], [rho, 1.]]

c = 1.
alpha  = 1.
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
maxiter = 3500

init = np.ones(loss.dim)
loss = FourierGaussLossFunction(mu, sigma, alpha, c)
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})



#r to specify how many times to repeat and n number of times the function is called
%timeit res = minimize(loss.objective, init, jac=loss.objective_jac, constraints=cons, method='SLSQP',options={'maxiter': maxiter})
print(res)

11.8 s ± 1.09 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
     fun: -0.2855887897197693
     jac: array([1., 1.])
 message: 'Optimization terminated successfully'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([-0.1427944 , -0.14279439])
